In [8]:
!pip uninstall -y scipy numpy tensorflow tensorflow-addons
!pip install numpy==1.23.5
!pip install scipy==1.10.1
!pip install tensorflow==2.12.0
!pip install tensorflow-addons==0.20.0
!pip install pandas

Found existing installation: scipy 1.15.3
Uninstalling scipy-1.15.3:
  Successfully uninstalled scipy-1.15.3
Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
Found existing installation: tensorflow 2.18.0
Uninstalling tensorflow-2.18.0:
  Successfully uninstalled tensorflow-2.18.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 78.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cvxpy 1.6.5 requires scipy>=1.11.0, which is not installed.
jaxlib 0.5.1 requires scipy>=1.11.1, which is not installed.
jax 0.5.2 requires scipy>=1.11.1, which is not installed.
mizani 0.13.5 requires scipy>=1.8.0, which is not installed.
cuml-cu12 25.2.1 requires scipy>=1.8.0, which is not installed.
mlxtend 0.23.4 requires scipy>=1.2.1, which is not installed.
arviz 0.21.0 requires scipy>=1.9.0, which

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.1/34.1 MB 20.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dopamine-rl 4.1.2 requires tensorflow>=2.2.0, which is not installed.
cvxpy 1.6.5 requires scipy>=1.11.0, but you have scipy 1.10.1 which is incompatible.
jaxlib 0.5.1 requires numpy>=1.25, but you have numpy 1.23.5 which is incompatible.
jaxlib 0.5.1 requires scipy>=1.11.1, but you have scipy 1.10.1 which is incompatible.
jax 0.5.2 requires numpy>=1.25, but you have numpy 1.23.5 which is incompatible.
jax 0.5.2 requires scipy>=1.11.1, but you have scipy 1.10.1 which is incompatible.
chex 0.1.89 requires numpy>=1.24.1, but you have numpy 1.23.5 which is incompatible.
scikit-image 0.25.2 requires numpy>=1.24, but you have numpy 1.23.5 which is incompatible.
s

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 591.0/591.0 kB 7.6 MB/s eta 0:00:00
  Attempting uninstall: typeguard
    Found existing installation: typeguard 4.4.2
    Uninstalling typeguard-4.4.2:
      Successfully uninstalled typeguard-4.4.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
inflect 7.5.0 requires typeguard>=4.0.1, but you have typeguard 2.13.3 which is incompatible.


In [1]:
import os
import re
import pandas as pd
import numpy as np
from collections import defaultdict
import zipfile
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, precision_recall_fscore_support
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense, Dropout, Concatenate, Embedding, Flatten, LSTM, Bidirectional
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
def extract_coref_data(coref_file_path):
    with open(coref_file_path, 'r', encoding='utf-8') as f:
        content = f.read()

    text_match = re.search(r'<TEXT[^>]*>(.*?)</TEXT>', content, re.DOTALL)
    if not text_match:
        return None, {}

    text = text_match.group(1)

    coref_chains = defaultdict(list)
    coref_tags = re.finditer(r'<COREF ID="(\d+)" TYPE="([^"]+)"[^>]*>(.*?)</COREF>', text, re.DOTALL)

    for match in coref_tags:
        coref_id = match.group(1)
        coref_type = match.group(2)
        mention_text = match.group(3)

        clean_text = re.sub(r'<[^>]+>', '', mention_text)

        coref_chains[coref_id].append({
            'text': clean_text,
            'type': coref_type,
            'start': match.start(),
            'end': match.end()
        })

    return text, dict(coref_chains)

In [4]:
def extract_onf_data(onf_file_path):
    with open(onf_file_path, 'r', encoding='utf-8') as f:
        content = f.read()

    plain_sentences = []
    plain_matches = re.finditer(r'Plain sentence:\n---------------\n(.*?)Treebanked sentence:', content, re.DOTALL)
    for match in plain_matches:
        plain_sentences.append(match.group(1).strip())

    treebanked_sentences = []
    tree_matches = re.finditer(r'Treebanked sentence:\n--------------------\n(.*?)Tree:', content, re.DOTALL)
    for match in tree_matches:
        treebanked_sentences.append(match.group(1).strip())

    all_tokens = []
    leaves_sections = re.finditer(r'Leaves:\n-------\n(.*?)(?=---------|$)', content, re.DOTALL)

    sentence_idx = 0
    for leaves_section in leaves_sections:
        leaves_text = leaves_section.group(1)

        token_matches = re.finditer(r'(\d+)\s+(\S+)\s+(.*?)(?=\n\s+\d+|\n\S|$)', leaves_text, re.DOTALL)

        for match in token_matches:
            token_id = int(match.group(1))
            token = match.group(2)
            token_metadata = match.group(3).strip()

            coref_match = re.search(r'coref:\s+IDENT\s+(\d+)', token_metadata)
            coref_id = coref_match.group(1) if coref_match else None

            named_entity_match = re.search(r'name:\s+(\S+)', token_metadata)
            named_entity = named_entity_match.group(1) if named_entity_match else None

            sense_match = re.search(r'sense:\s+(\S+)', token_metadata)
            sense = sense_match.group(1) if sense_match else None

            current_sentence = ""
            if sentence_idx < len(plain_sentences):
                current_sentence = plain_sentences[sentence_idx]

            all_tokens.append({
                'token_id': token_id,
                'token': token,
                'sentence_idx': sentence_idx,
                'sentence': current_sentence,
                'coref_id': coref_id,
                'named_entity': named_entity,
                'sense': sense,
                'metadata': token_metadata
            })

        sentence_idx += 1

    return all_tokens

In [5]:
def process_dataset(root_dir):
    all_documents = []

    for dirpath, dirnames, filenames in os.walk(root_dir):
        for filename in filenames:
            if filename.endswith('.coref'):
                doc_id = filename[:-6]
                coref_path = os.path.join(dirpath, filename)
                onf_path = os.path.join(dirpath, f"{doc_id}.onf")

                if os.path.exists(onf_path):
                    text, coref_chains = extract_coref_data(coref_path)
                    tokens_info = extract_onf_data(onf_path)

                    if text and tokens_info:
                        all_documents.append({
                            'doc_id': doc_id,
                            'text': text,
                            'coref_chains': coref_chains,
                            'tokens': tokens_info
                        })

    return all_documents

In [6]:
def process_ontonotes_to_dataframes(root_dir):
    documents_data = []
    tokens_data = []
    coref_mentions_data = []
    coref_chains_data = []

    for dirpath, dirnames, filenames in os.walk(root_dir):
        for filename in filenames:
            if filename.endswith('.coref'):
                doc_id = filename[:-6]
                coref_path = os.path.join(dirpath, filename)
                onf_path = os.path.join(dirpath, f"{doc_id}.onf")

                if os.path.exists(onf_path):
                    text, coref_chains = extract_coref_data(coref_path)

                    tokens_info = extract_onf_data(onf_path)

                    if text and tokens_info:
                        doc_info = {
                            'doc_id': doc_id,
                            'path': dirpath,
                            'num_tokens': len(tokens_info),
                            'num_coref_chains': len(coref_chains),
                        }
                        documents_data.append(doc_info)

                        for token in tokens_info:
                            token_info = {
                                'doc_id': doc_id,
                                **token
                            }
                            tokens_data.append(token_info)

                        for chain_id, mentions in coref_chains.items():
                            for mention_idx, mention in enumerate(mentions):
                                mention_info = {
                                    'doc_id': doc_id,
                                    'chain_id': chain_id,
                                    'mention_idx': mention_idx,
                                    'mention_text': mention['text'],
                                    'mention_type': mention['type'],
                                    'start_pos': mention['start'],
                                    'end_pos': mention['end']
                                }
                                coref_mentions_data.append(mention_info)

                            chain_info = {
                                'doc_id': doc_id,
                                'chain_id': chain_id,
                                'num_mentions': len(mentions),
                                'mentions_text': [m['text'] for m in mentions]
                            }
                            coref_chains_data.append(chain_info)

    df_documents = pd.DataFrame(documents_data)
    df_tokens = pd.DataFrame(tokens_data)
    df_mentions = pd.DataFrame(coref_mentions_data)
    df_chains = pd.DataFrame(coref_chains_data)

    return df_documents, df_tokens, df_mentions, df_chains

In [7]:
data_dir = '/content/drive/MyDrive/ann'
output_dir = '/content/drive/MyDrive/ontonotes_dataframes'

df_documents, df_tokens, df_mentions, df_chains = process_ontonotes_to_dataframes(data_dir)


In [9]:
df_mentions.tail(5)

,doc_id,chain_id,mention_idx,mention_text,mention_type,start_pos,end_pos
35827,ann_0178,29590,0,كارول كالين,APPOS,17079,17144
35828,ann_0178,29599,0,-المُسْتَشارَةُ السِياسِيَّةُ فِي السِفارَةِ,APPOS,17232,17332
35829,ann_0178,29599,1,آن بودين,APPOS,17333,17395
35830,ann_0178,29614,0,المُحادَثاتِ,IDENT,18009,18060
35831,ann_0178,29614,1,-ها,IDENT,18083,18135


In [10]:
def remove_tashkeel(w):
    p_tashkeel = re.compile(r'[\u0617-\u061A\u064B-\u0652]')
    w = re.sub(p_tashkeel, "", w)
    return w

In [11]:
def prepare_dataset(df_mentions, df_tokens, max_distance=50, max_pairs_per_doc=10000):
    feature_data = []

    for doc_id in tqdm(df_mentions['doc_id'].unique(), desc="Traitement des documents"):
        doc_mentions = df_mentions[df_mentions['doc_id'] == doc_id].sort_values('start_pos').reset_index(drop=True)
        doc_tokens = df_tokens[df_tokens['doc_id'] == doc_id]

        pairs_count = 0

        for i, mention1 in doc_mentions.iterrows():
            if pairs_count >= max_pairs_per_doc:
                break

            for j, mention2 in doc_mentions.iloc[i+1:].iterrows():
                if mention2['start_pos'] - mention1['end_pos'] > max_distance:
                    continue

                is_coreferent = int(mention1['chain_id'] == mention2['chain_id'] and mention1['chain_id'] is not None)

                m1_text = remove_tashkeel(mention1['mention_text'])
                m2_text = remove_tashkeel(mention2['mention_text'])

                token_distance = mention2['start_pos'] - mention1['end_pos']

                mention_types = {
                    'm1_type': mention1['mention_type'],
                    'm2_type': mention2['mention_type']
                }

                feature_data.append({
                    'doc_id': doc_id,
                    'mention1_id': i,
                    'mention2_id': j,
                    'm1_text': m1_text,
                    'm2_text': m2_text,
                    'token_distance': token_distance,
                    'm1_type': mention_types['m1_type'],
                    'm2_type': mention_types['m2_type'],
                    'is_coreferent': is_coreferent
                })

                pairs_count += 1

    df_pairs = pd.DataFrame(feature_data)

    return df_pairs


In [12]:
df_pairs= prepare_dataset(df_mentions,df_tokens)

Traitement des documents:   0%|          | 0/447 [00:00<?, ?it/s]

In [13]:
df_pairs.head()

,doc_id,mention1_id,mention2_id,m1_text,m2_text,token_distance,m1_type,m2_type,is_coreferent
0,ann_0205,0,1,الكؤوس الإفريقية,الزمالك,8,IDENT,IDENT,0
1,ann_0205,1,2,الزمالك,-الأهلي,5,IDENT,IDENT,0
2,ann_0205,2,3,-الأهلي,-الترجي,5,IDENT,IDENT,0
3,ann_0205,3,4,-الترجي,-الرجاء,5,IDENT,IDENT,0
4,ann_0205,4,5,-الرجاء,دوري الأبطال,1,IDENT,IDENT,0


In [32]:
def prepare_features_for_dl(df_pairs):

    m1_type_encoder = LabelEncoder()
    m2_type_encoder = LabelEncoder()

    df_pairs['m1_type_encoded'] = m1_type_encoder.fit_transform(df_pairs['m1_type'].fillna('NONE'))
    df_pairs['m2_type_encoded'] = m2_type_encoder.fit_transform(df_pairs['m2_type'].fillna('NONE'))

    scaler = StandardScaler()
    distance_features = ['token_distance']
    df_pairs[distance_features] = scaler.fit_transform(df_pairs[distance_features])

    train_data, test_data = train_test_split(df_pairs, test_size=0.2, random_state=42,
                                            stratify=df_pairs['is_coreferent'])

    return train_data, test_data, m1_type_encoder, m2_type_encoder, scaler

train_data, test_data, m1_type_encoder, m2_type_encoder, scaler = prepare_features_for_dl(df_pairs)

Suppression des caractéres spéciaux et normalisation des espaces

In [33]:
def preprocess_text_for_arabert(text):
    if isinstance(text, str):
        text = re.sub(r'[^\u0600-\u06FF\s]', ' ', text)
        text = re.sub(r'\s+', ' ', text).strip()
        return text
    return ""


In [55]:
print("Distribution des classes dans les données d'entraînement:")
print(df_pairs['is_coreferent'].value_counts())
print("Pourcentage de coréférences:", df_pairs['is_coreferent'].mean())

Distribution des classes dans les données d'entraînement:
is_coreferent
0    17010
1     7687
Name: count, dtype: int64
Pourcentage de coréférences: 0.311252378831437


In [34]:
    max_len = 50
    vocab_size=len(df_pairs)
    embedding_dim = 300
    m1_type_classes=10
    m2_type_classes=10

    input_m1_text = Input(shape=(max_len,), name='input_m1_text')
    input_m2_text = Input(shape=(max_len,), name='input_m2_text')
    input_m1_type = Input(shape=(1,), name='input_m1_type')
    input_m2_type = Input(shape=(1,), name='input_m2_type')
    input_distances = Input(shape=(1,), name='input_distances')

    embedding_layer = Embedding(input_dim=vocab_size, output_dim=embedding_dim)

    m1_embedding = embedding_layer(input_m1_text)
    m2_embedding = embedding_layer(input_m2_text)

    m1_embedding = Bidirectional(LSTM(128))(m1_embedding)
    m2_embedding = Bidirectional(LSTM(128))(m2_embedding)

    m1_type_embedding = Embedding(input_dim=m1_type_classes, output_dim=50,
                                     input_length=1)(input_m1_type)
    m2_type_embedding = Embedding(input_dim=m2_type_classes, output_dim=50,
                                     input_length=1)(input_m2_type)

    m1_type_embedding = Flatten()(m1_type_embedding)
    m2_type_embedding = Flatten()(m2_type_embedding)

    merged = Concatenate()([m1_embedding, m2_embedding, m1_type_embedding, m2_type_embedding, input_distances])

    x = Dense(512, activation='relu')(merged)
    x = Dropout(0.3)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.3)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.2)(x)

    output = Dense(1, activation='sigmoid', name='output')(x)

    model = Model(
        inputs=[input_m1_text, input_m2_text, input_m1_type, input_m2_type, input_distances],
        outputs=output
    )

    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
    )

    train_texts_m1 = [preprocess_text_for_arabert(t) for t in train_data['m1_text']]
    train_texts_m2 = [preprocess_text_for_arabert(t) for t in train_data['m2_text']]
    test_texts_m1 = [preprocess_text_for_arabert(t) for t in test_data['m1_text']]
    test_texts_m2 = [preprocess_text_for_arabert(t) for t in test_data['m2_text']]

    tokenizer = Tokenizer(oov_token="<OOV>")
    tokenizer.fit_on_texts(train_texts_m1 + train_texts_m2)

    vocab_size = len(tokenizer.word_index) + 1
    m1_type_classes = len(m1_type_encoder.classes_)
    m2_type_classes = len(m2_type_encoder.classes_)

    max_len = 50
    train_seq_m1 = pad_sequences(tokenizer.texts_to_sequences(train_texts_m1), maxlen=max_len, padding='post')
    train_seq_m2 = pad_sequences(tokenizer.texts_to_sequences(train_texts_m2), maxlen=max_len, padding='post')
    test_seq_m1 = pad_sequences(tokenizer.texts_to_sequences(test_texts_m1), maxlen=max_len, padding='post')
    test_seq_m2 = pad_sequences(tokenizer.texts_to_sequences(test_texts_m2), maxlen=max_len, padding='post')

    train_inputs = {
        'input_m1_text': train_seq_m1,
        'input_m2_text': train_seq_m2,
        'input_m1_type': train_data['m1_type_encoded'].values.reshape(-1, 1),
        'input_m2_type': train_data['m2_type_encoded'].values.reshape(-1, 1),
        'input_distances': train_data[['token_distance']].values
    }

    train_targets = train_data['is_coreferent'].values

    history = model.fit(
        train_inputs,
        train_targets,
        batch_size=16,
        epochs=10,
        verbose=1,
        validation_split=0.2
    )

Epoch 1/10
988/988 [==============================] - 445s 443ms/step - loss: 0.3990 - accuracy: 0.8120 - precision_1: 0.7017 - recall_1: 0.6806 - val_loss: 0.3451 - val_accuracy: 0.8454 - val_precision_1: 0.7951 - val_recall_1: 0.6979
Epoch 2/10
988/988 [==============================] - 352s 356ms/step - loss: 0.2891 - accuracy: 0.8736 - precision_1: 0.7915 - recall_1: 0.8019 - val_loss: 0.3432 - val_accuracy: 0.8603 - val_precision_1: 0.7775 - val_recall_1: 0.7910
Epoch 3/10
988/988 [==============================] - 341s 345ms/step - loss: 0.2184 - accuracy: 0.9047 - precision_1: 0.8421 - recall_1: 0.8509 - val_loss: 0.3968 - val_accuracy: 0.8598 - val_precision_1: 0.7692 - val_recall_1: 0.8044
Epoch 4/10
988/988 [==============================] - 340s 344ms/step - loss: 0.1824 - accuracy: 0.9203 - precision_1: 0.8699 - recall_1: 0.8726 - val_loss: 0.5035 - val_accuracy: 0.8378 - val_precision_1: 0.8132 - val_recall_1: 0.6420
Epoch 5/10
988/988 [==============================] - 34

In [35]:
import pickle
model.save("/content/drive/MyDrive/Coref.h5")

with open('/content/drive/MyDrive/tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)
with open('/content/drive/MyDrive/m1_type_encoder.pkl', 'wb') as f:
    pickle.dump(m1_type_encoder, f)
with open('/content/drive/MyDrive/m2_type_encoder.pkl', 'wb') as f:
    pickle.dump(m2_type_encoder, f)
with open('/content/drive/MyDrive/scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

In [18]:
y_pred_proba = model.predict(test_inputs)
y_pred = (y_pred_proba > 0.5).astype(int).flatten()
print("\nRapport de classification:")
print(classification_report(test_targets, y_pred))

155/155 [==============================] - 15s 89ms/step

Rapport de classification:
              precision    recall  f1-score   support

           0       0.91      0.89      0.90      3402
           1       0.77      0.81      0.79      1538

    accuracy                           0.86      4940
   macro avg       0.84      0.85      0.84      4940
weighted avg       0.87      0.86      0.86      4940



In [37]:
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import re

model_name = "CAMeL-Lab/bert-base-arabic-camelbert-msa-ner"
ner_tokenizer = AutoTokenizer.from_pretrained(model_name)
ner_model = AutoModelForTokenClassification.from_pretrained(model_name)
ner_pipeline = pipeline("ner", model=ner_model, tokenizer=ner_tokenizer, aggregation_strategy="simple")


Some weights of the model checkpoint at CAMeL-Lab/bert-base-arabic-camelbert-msa-ner were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


In [38]:
with open('/content/drive/MyDrive/tokenizer.pkl', 'rb') as f:
    tokenizer = pickle.load(f)
with open('/content/drive/MyDrive/m1_type_encoder.pkl', 'rb') as f:
    m1_type_encoder = pickle.load(f)
with open('/content/drive/MyDrive/m2_type_encoder.pkl', 'rb') as f:
    m2_type_encoder = pickle.load(f)
with open('/content/drive/MyDrive/scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

In [39]:
def safe_transform(label, encoder):
    if label in encoder.classes_:
        return encoder.transform([label])[0]
    else:
        return encoder.transform([encoder.classes_[0]])[0]


In [ ]:
model_name = "CAMeL-Lab/bert-base-arabic-camelbert-msa-ner"
ner_tokenizer = AutoTokenizer.from_pretrained(model_name)
ner_model = AutoModelForTokenClassification.from_pretrained(model_name)
ner_pipeline = pipeline("ner", model=ner_model, tokenizer=ner_tokenizer, aggregation_strategy="simple")

def fix_scaler_warning(distance, scaler):
    import pandas as pd
    distance_df = pd.DataFrame([[distance]], columns=['token_distance'])
    distance_scaled = scaler.transform(distance_df)[0][0]
    return distance_scaled

def predict_coreference_fixed(sentence, model, max_len=50):

    ner_results = ner_pipeline(sentence)
    mentions = []
    for r in ner_results:
        mentions.append({
            'text': r['word'],
            'start': r['start'],
            'end': r['end'],
            'label': r['entity_group']
        })

    print(f"Mentions trouvées: {len(mentions)}")
    for m in mentions:
        print(f"- {m['text']} ({m['label']}) at {m['start']}-{m['end']}")

    if len(mentions) < 2:
        return []

    from itertools import combinations
    mention_pairs = list(combinations(mentions, 2))

    all_inputs = {
        'input_m1_text': [],
        'input_m2_text': [],
        'input_m1_type': [],
        'input_m2_type': [],
        'input_distances': []
    }

    for m1, m2 in mention_pairs:
        text1 = preprocess_text_for_arabert(m1['text'])
        text2 = preprocess_text_for_arabert(m2['text'])

        seq1 = tokenizer.texts_to_sequences([text1])
        seq2 = tokenizer.texts_to_sequences([text2])

        if not seq1 or not seq1[0]:
            seq1 = [[1]]
        if not seq2 or not seq2[0]:
            seq2 = [[1]]

        seq1_padded = pad_sequences(seq1, maxlen=max_len, padding='post')[0]
        seq2_padded = pad_sequences(seq2, maxlen=max_len, padding='post')[0]

        type1 = safe_transform(m1["label"], m1_type_encoder)
        type2 = safe_transform(m2["label"], m2_type_encoder)

        distance = abs(m2['start'] - m1['end'])
        distance_scaled = fix_scaler_warning(distance, scaler)

        all_inputs['input_m1_text'].append(seq1_padded)
        all_inputs['input_m2_text'].append(seq2_padded)
        all_inputs['input_m1_type'].append([type1])
        all_inputs['input_m2_type'].append([type2])
        all_inputs['input_distances'].append([distance_scaled])

    for key in all_inputs:
        all_inputs[key] = np.array(all_inputs[key])

    predictions = model.predict(all_inputs, verbose=0)

    coref_pairs = []
    threshold = 0.5

    for i, (m1, m2) in enumerate(mention_pairs):
        score = predictions[i][0]
        print(f"Paire: '{m1['text']}' vs '{m2['text']}' -> Score: {score:.4f}")

        if score > threshold:
            coref_pairs.append({
                'mention1': m1,
                'mention2': m2,
                'score': float(score)
            })

    return coref_pairs

def test_real_coreference_examples(model):

    test_cases = [
        {
            'sentence': "أحمد طالب ذكي. هو يدرس في الجامعة.",
            'expected': "أحمد et هو devraient être coréférents"
        },

        {
            'sentence': "زار محمد المتحف. محمد كان معجباً بالمعروضات.",
            'expected': "Les deux occurrences de محمد devraient être coréférentes"
        },

        {
            'sentence': "الطبيب فحص المريض. قال الطبيب إن الحالة جيدة.",
            'expected': "Les deux occurrences de الطبيب devraient être coréférentes"
        },

        {
            'sentence': "فاطمة مهندسة ماهرة. هي تعمل في شركة كبيرة.",
            'expected': "فاطمة et هي devraient être coréférents"
        },

        {
            'sentence': "جاء الأستاذ إلى الصف. المعلم شرح الدرس بوضوح.",
            'expected': "الأستاذ et المعلم pourraient être coréférents"
        }
    ]

    print("=" * 60)
    print("TEST DE CORÉFÉRENCE AVEC EXEMPLES RÉELS")
    print("=" * 60)

    for i, test_case in enumerate(test_cases, 1):
        print(f"\n--- Test {i} ---")
        print(f"Phrase: {test_case['sentence']}")
        print(f"Attendu: {test_case['expected']}")
        print("-" * 40)

        results = predict_coreference_fixed(test_case['sentence'], model)

        if results:
            print("Coréférences détectées:")
            for r in results:
                print(f"  '{r['mention1']['text']}' ↔ '{r['mention2']['text']}' (score: {r['score']:.3f})")
        else:
            print("Aucune coréférence détectée")

        print()

test_real_coreference_examples(model)